In [2]:
from sqlalchemy import create_engine, text
import pandas as pd
from urllib.parse import quote


# df = pd.read_json(r"C:\Users\mark.ding\big-data-ai-integration-platform\sparkVolumnTest\企業資訊_全_2024_10_24.json")
df = pd.read_json(r"C:\Users\m7812\Desktop\Mark_info_git\big-data-ai-integration-platform\sparkVolumnTest\whole_corp_2024_10_24.json")

for column in df.columns:
    df[column] = df[column].apply(lambda x : ','.join(map(str,x)) if isinstance(x, list) else x)
for i in df.columns:
    df[i] = df[i].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)



In [6]:
import pandas as pd
from sqlalchemy import create_engine, text
import urllib.parse, os, csv

user = "root"
password = "!QAZ2wsx"
host = "localhost"
port = 3307
db   = "whole_corp"
tbl  = "whole_corp"

encoded_pw = urllib.parse.quote_plus(password)
# IMPORTANT: allow_local_infile=1 for mysqlconnector (or local_infile=1 for PyMySQL)
db_url = f"mysql+mysqlconnector://{user}:{encoded_pw}@{host}:{port}/{db}?allow_local_infile=1"

engine = create_engine(db_url, pool_pre_ping=True)

# 1) Ensure table exists with the right schema.
#    If you don't have it yet, create it once with to_sql but with zero rows
#    (or write a CREATE TABLE manually for precise types).
#    Example (create empty table once):
# df.head(0).to_sql(tbl, engine, if_exists="replace", index=False)

# 2) Write a clean TSV to disk (much safer than CSV for commas/quotes)
tmp_path = "/tmp/whole_corp_load.tsv"  # on Windows: r"C:\temp\whole_corp_load.tsv"
os.makedirs(os.path.dirname(tmp_path), exist_ok=True)

# Write TSV: represent NULLs as \N (MySQL understands \N as NULL if not quoted)
df.to_csv(
    tmp_path,
    sep="\t",
    index=False,
    na_rep="\\N",
    quoting=csv.QUOTE_NONE,
    escapechar="\\",
)

# 3) Bulk load
with engine.begin() as conn:
    # optional but helps: disable checks while loading
    conn.execute(text("SET FOREIGN_KEY_CHECKS=0"))
    conn.execute(text("SET UNIQUE_CHECKS=0"))
    # If you have heavy indexes on the table, consider dropping them before and recreating after.

    # LOAD DATA (LOCAL needs allow_local_infile=1)
    conn.execute(text(f"""
        LOAD DATA LOCAL INFILE :path
        INTO TABLE {tbl}
        FIELDS TERMINATED BY '\t'
        ESCAPED BY '\\\\'
        LINES TERMINATED BY '\n'
        IGNORE 1 LINES
    """), {"path": tmp_path})

    conn.execute(text("SET UNIQUE_CHECKS=1"))
    conn.execute(text("SET FOREIGN_KEY_CHECKS=1"))


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1512083 entries, 0 to 1512082
Data columns (total 36 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   統一編號      1512083 non-null  object
 1   公司名稱      1512083 non-null  object
 2   負責人       1344752 non-null  object
 3   登記地址      1498651 non-null  object
 4   公司狀態      862655 non-null   object
 5   資本額       1148835 non-null  object
 6   登記機關      862654 non-null   object
 7   設立日期      859885 non-null   object
 8   營業項目及代碼表  1310797 non-null  object
 9   財政營業項目    862655 non-null   object
 10  類別        862655 non-null   object
 11  縣市名稱      1498651 non-null  object
 12  區域名稱      1496807 non-null  object
 13  縣市區域      1487973 non-null  object
 14  產品        750831 non-null   object
 15  產品說明      750831 non-null   object
 16  實收資本總額    86421 non-null    object
 17  上市櫃_基本資料  1828 non-null     object
 18  官網        57726 non-null    object
 19  公司位置      71637 non-null    object
 20  聯絡人    